In [1]:
import numpy as np
import matplotlib.pyplot as plt
from Helper import *

In [2]:
#GRAPHS#
SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
# 0 - male, 1 - female 
gender = [1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, # player 0 - 16
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, # 17 - 32
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, # 33 - 48
         0, 0, 0, 0, 1, 1, 1, 1, 0, 0 ,0 ,0, 0, 0, 0, 0 ] # 49 - 64 

gen = [0,1,1,1,0,0,0,0,1,1,1,1,0,1,0,0]
colours = ["blue","red","red","red","blue","blue","blue","blue","red","red","red","red","blue","red","blue","blue"]
armLength = [28,23.5,27,27, 28, 27, 0, 26, 23, 26, 24, 26, 27, 24, 0, 29]
vrLength = [4.94, 3.90, 2.49, 3.39, 5.27, 5.09, 7.19, 4.14, 6.20, 3.00, 4.21, 4.90, 5.45, 2.87, 6.95, 2.70]
realLength = [ a * b / 100 for (a,b) in zip(armLength,vrLength) ]

weightPercentageMale = [0.551, 0.0325, 0.0325, 0.1668, 0.1668, 0.0252, 0.0252]
weightPercentageFemale = [0.532, 0.029, 0.029, 0.1843, 0.1843, 0.0207, 0.0207]

playersTraining = [2, 7, 11, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62]
playersBaseline = [4, 9, 13, 16, 19, 24, 28, 31, 36, 39, 43, 48, 51, 56, 60, 63]
playersOptimal  = [3, 8, 12, 15, 20, 23, 27, 32, 35, 40, 44, 47, 52, 55, 59, 64]

path = "E:\\Andreea\\Master\\Thesis\\Git\\TheOne\\SmartuistWrapper\\tapResultParticipants.csv"
pathH = "C:\\Users\\huber\\OneDrive\\Dokumenty\\thesis-hubert\\unity\\SmartuistWrapper\\tapResultParticipants.csv"

In [7]:
def getStatisticsPhase3(results, players, limitLessons = 2):
    mvmTime = [] 
    gain = []
    efforBodyParts = []
    distBodyParts = []
    throughput = []
    for p in players:
        tmp = results[p]
        time = 0
        count = 0
        th = 0
        gain.append(getGainPerPlayer(p, results))
        effort, dist = EffortPerBodyPartPerPlayerSummed(p, results,10, limitLessons)
        efforBodyParts.append(effort)
        distBodyParts.append(dist)
        for words in tmp:
            if int(words[1]) < 2:
                time += float(words[10])
                th += float(words[11]) / (float(words[10]) / 1000) # recompute as id/time (not normalized id)
                count += 1
        throughput.append(th/count)        
        mvmTime.append(time/1000) # time in seconds
    return (mvmTime, gain, np.array(efforBodyParts), np.array(distBodyParts), throughput)

# computes the gain for the player (from effort per body part and throughput)
def getGainPerPlayer(p, results):
    gain = [] 
    batchSize = 10
    nrLessons = 2
    playerResults = results[p]
    for l in range(nrLessons):
        stupidTh = 0 
        sumTh = 0
        bodyParts =[ 0 for i in range(7)]
        for res in playerResults: #results for a repetition  
            if(l == int(res[1])):
                for i in range (7):
                    bodyParts[i] += 1 / float(res[21+i])
                sumTh += float(res[13])
                stupidTh += sumTh
        
        bodyParts = [ x / batchSize for x in bodyParts]
        gainEffort = sum(bodyParts) / len(bodyParts)
        gainTh = stupidTh / batchSize
        gain.append((gainEffort + gainTh) / 2)
    
    return np.average(gain)

# get effort and distance per body parts
def EffortPerBodyPartPerPlayerSummed(p, results, batchSize, limitLessons):
    effortBodyParts = [ 0 for i in range(7)]
    distBodyParts = [ 0 for i in range(7)]
    tmp = results[p]
    counter = 0
    for words in tmp:
        if int(words[1]) < limitLessons:
            for i in range (7):
                effortBodyParts[i] += float(words[21+i])
                distBodyParts[i] += float(words[14+i])

            counter = counter + 1
            if counter == batchSize:
                counter = 0
            if counter == 0:
                for i in range(7):
                    effortBodyParts[i] /= batchSize
                    distBodyParts[i] /= batchSize
    return (effortBodyParts, distBodyParts)

# a list with the results at idx for the given player    
def getResultPerPlayer(p, results, idx, unique ):
    tmp = results[p]
    values = []
    for words in tmp:
        values.append(float(words[idx]))
    if (unique == True):
        values =  [x for i, x in enumerate(values) if i == values.index(x)]
    return values

# sort list1 - use the indexes of the sorted list1 to rearrage list2    
def getCombinedPlot(list1, list2):
    idx   = np.argsort(list1)
    list1 = np.array(list1)[idx]
    list2 = np.array(list2)[idx]  
    return (list1, list2)

def EffortPerBodyPartPerPlayer(p, results):
    bodyParts =[ [0]*1 for i in range(7)]
    distBodyParts = [[0] * 1 for i in range (7)]
    tmp = results[p]
    counter = 0
    for words in tmp:            
        for i in range (7):
            bodyParts[i][bodyParts[i].index(bodyParts[i][-1])] += float(words[21+i])
            distBodyParts[i][distBodyParts[i].index(distBodyParts[i][-1])] += float(words[14+i])
        counter = counter + 1
        if counter == 10:
            counter = 0
        if counter == 0:
            for i in range(7):
                bodyParts[i][bodyParts[i].index(bodyParts[i][-1])] /= 10
                bodyParts[i].append(0)
                distBodyParts[i][distBodyParts[i].index(distBodyParts[i][-1])] /= 10
                distBodyParts[i].append(0)
    return (bodyParts, distBodyParts)

In [10]:
results = readFromFile(path, 0)
mvmTimeBaseline, gainBaseline, effortBaseline, distBaseline, thBaseline = getStatisticsPhase3(results, playersBaseline)
mvmTimeOptimal, gainOptimal, effortOptimal, distOptimal, thOptimal  = getStatisticsPhase3(results, playersOptimal)

In [37]:
# statistics for phase 2
from scipy.stats import pearsonr, spearmanr, kendalltau, ttest_rel, wilcoxon, normaltest, mannwhitneyu

def isNormal(x):
    k2, p = normaltest(x)
    alpha = 1e-3
    #print("p = {:g}".format(p))
    p = 3.27207e-11
    if p < alpha:  # null hypothesis: x comes from a normal distribution
        #rint("The null hypothesis can be rejected")
        return False
    else:
       #print("The null hypothesis cannot be rejected")
        return True

# wicloxen is not-parametric (can be used when the data is not uniformly distributed and there are outliners)
def findCorelationPairs(data1, data2, label=""):
    print(label)

   # stats, p = ttest_rel(data1, data2)   
   # print('Paired ttest p-value: %.7f' % p)
    
    # calculate kendall's correlation
   # coef, p = kendalltau(data1, data2)
   # print('Kendall correlation p-value: %.7f' % p) # for continuos independent var  
    
    isNormal(data1)
    isNormal(data2)
    
   # if(isNormal(data1) and isNormal(data2)):
    print("Data normaly distributed")
    stats, p = ttest_rel(data1, data2)    
    print('Paired t test  p-value / coef: %.7f' % p, stats) # for continuos independent var  
  #  else:    
    print("Data not normaly distributed")
    stats, p = wilcoxon(data1, data2)    
    print('Paired wilcoxon  p-value .coef : %.7f' % p, stats) # for continuos independent var  

    stats, p = mannwhitneyu(data1, data2)    
   # print('mann-whitney u test: p-value .coef : %.7f' % p, stats) # for continuos independent var  
             
    print(" ")
    
    
findCorelationPairs(mvmTimeBaseline, mvmTimeOptimal, " Movement ")
findCorelationPairs(gainBaseline, gainOptimal, " Fitness ")
findCorelationPairs(thBaseline, thOptimal, " Throughput ")

def stats(data, lable):
    median = np.median(data)
    upper_quartile = np.percentile(data, 75)
    lower_quartile = np.percentile(data, 25)
    iqr = upper_quartile - lower_quartile
    print(lable, "{:.2f} & {:.2f}".format(median, iqr))
    
stats(mvmTimeBaseline, "mvmtimebaseline")
stats(mvmTimeOptimal,  "mvmTimeOptimal")
stats(gainBaseline, "gainBaseline")
stats(gainOptimal, "gainOptimal")
stats(thBaseline, "thBaseline")
stats(thOptimal, "thOptimal")

# bodyParts = ["Body", "Left Upper Arm", "Right Upper Arm" , "Right Foot" , "Left Foot", "Left Hand" , "Right Hand", ]
# for i in range(7):   
#     findCorelationPairs(effortBaseline[:, i], effortOptimal[: ,i], " Effort " + bodyParts[i])
#     findCorelationPairs(distBaseline[:, i], distOptimal[: ,i], " Dist " + bodyParts[i])

 Movement 
Data normaly distributed
Paired t test  p-value / coef: 0.0000029 7.228725753973966
Data not normaly distributed
Paired wilcoxon  p-value .coef : 0.0004378 0.0
 
 Fitness 
Data normaly distributed
Paired t test  p-value / coef: 0.0000001 -9.811417278680844
Data not normaly distributed
Paired wilcoxon  p-value .coef : 0.0004378 0.0
 
 Throughput 
Data normaly distributed
Paired t test  p-value / coef: 0.0000001 -9.405450312691135
Data not normaly distributed
Paired wilcoxon  p-value .coef : 0.0004378 0.0
 
mvmtimebaseline 47.91 & 11.56
mvmTimeOptimal 38.27 & 6.43
gainBaseline 9.11 & 1.61
gainOptimal 13.22 & 3.20
thBaseline 1.46 & 0.30
thOptimal 1.93 & 0.47
